## Calculate mAP of Custom Baseline model

This python worksheet calulates **mAp score** on a **validation data set** of **1600 records**

A **python library** called  **mean_average_precision** is used for the same  

In [ ]:
%tensorflow_version 2.x


In [ ]:
import tensorflow
print(tensorflow.__version__)

2.8.0


In [ ]:
import keras
print(keras.__version__)

2.8.0


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import sys
import numpy as np
import seaborn as sns
import statistics as stats
sns.set(color_codes=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#Last import allows multiple outputs from one cell
import warnings
# Initialize the random number generator
import random
random.seed(0)

#Additional libs
import matplotlib.patches as patches


In [ ]:
import os
import tensorflow 
from keras.models import Model
#from tensorflow.keras.optimizers import Adam, SGD
from keras.applications.vgg16 import VGG16, preprocess_input
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np
import matplotlib.image as img

#import tensorflow_addons as tfa

In [ ]:
# Useful Configuration/Setting

# suppress display of warnings
#warnings.filterwarnings('ignore')

# display all dataframe columns
pd.options.display.max_columns = None

# to set the limit to 3 decimals
pd.options.display.float_format = '{:.7f}'.format

# display all dataframe rows
pd.options.display.max_rows = None

#Setting to shows all entries in array displayed
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data_dir = Path('/content/drive/MyDrive/Capstone/Car Images/Car Images/Train Images')
test_data_dir = Path('/content/drive/MyDrive/Capstone/Car Images/Car Images/Test Images')

In [ ]:
import pandas as pd

#Lets read the Train Annotations

#Lets give our own column names as excel sheet does not have proper column names
columnNameList=['file_name','cor1','cor2','cor3','cor4','Image_Class']
trainAnno= pd.read_csv('/content/drive/MyDrive/Capstone/Annotations/Train Annotations.csv',names=columnNameList,header=0)
trainAnno.head(5)

,file_name,cor1,cor2,cor3,cor4,Image_Class
0,00001.jpg,39,116,569,375,14
1,00002.jpg,36,116,868,587,3
2,00003.jpg,85,109,601,381,91
3,00004.jpg,621,393,1484,1096,134
4,00005.jpg,14,36,133,99,106


In [ ]:
files = train_data_dir.rglob('*.jpg')
input=pd.DataFrame(files,columns =['a'])
input['a']=input['a'].astype(str)
df_train=input['a'].str.split('/',expand=True)
train=input.join(df_train)[['a',8]]
train.columns=('file_path','label')

In [ ]:
train.head()
len(train)

,file_path,label
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012


8144

In [ ]:
train.head()

,file_path,label
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012


In [ ]:
train['file_name']=train['file_path'].apply(lambda getFileName:getFileName.split('/')[9])

In [ ]:
train.head()

,file_path,label,file_name
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00482.jpg
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,05872.jpg
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,06576.jpg
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00719.jpg
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,04655.jpg


In [ ]:
trainAnno.head()

,file_name,cor1,cor2,cor3,cor4,Image_Class
0,00001.jpg,39,116,569,375,14
1,00002.jpg,36,116,868,587,3
2,00003.jpg,85,109,601,381,91
3,00004.jpg,621,393,1484,1096,134
4,00005.jpg,14,36,133,99,106


In [ ]:
# Lets add coordinate information to the above table
trainDF=pd.merge(train,trainAnno,on='file_name',how='inner')

In [ ]:
trainDF.head(5)

,file_path,label,file_name,cor1,cor2,cor3,cor4,Image_Class
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00482.jpg,80,91,589,429,184
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,05872.jpg,39,35,591,413,184
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,06576.jpg,64,347,629,577,184
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00719.jpg,118,93,380,317,184
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,04655.jpg,9,11,824,489,184


In [ ]:
from sklearn.model_selection import train_test_split
train_df,valid_df= train_test_split(trainDF, test_size=0.20, random_state=0, stratify=train['label'])

In [ ]:
#train_df,valid_df= train_test_split(valid_df, test_size=0.20, random_state=1, stratify=valid_df['label'])

In [ ]:
train_df.shape
valid_df.shape

(6515, 8)

(1629, 8)

In [ ]:
############################################################# Load the baseline Model #####################################################################################

In [ ]:
# Create reg model and load its weights
# Lets build the model

from keras.applications.mobilenet import MobileNet
from keras.models import Model,load_model
from keras.layers import Conv2D, Reshape,Flatten,Dense,Dropout,GlobalAveragePooling2D,BatchNormalization,Input
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

ALPHA = 1.0

def create_reg_od_model(trainable=True):
    regODModel = MobileNet(input_shape=(128, 128, 3), include_top=False, alpha=ALPHA) # Load pre-trained mobilenet

    # to freeze layers, except the new top layer, of course, which will be added below
    for layer in regODModel.layers:
        layer.trainable = trainable
    
    # Add new top layer for Regression Head
    regODModelOP = regODModel.layers[-1].output
    regODModelOP = Flatten()(regODModelOP)
    regODModelOP = Dense(units=4,name="box_output")(regODModelOP) # These are the 4 predicted coordinates of one BBox
    #boxPredicModel = Dense(units=4)(boxPredicModel) # These are the 4 predicted coordinates of one BBox 
    return Model(inputs=regODModel.input, outputs=regODModelOP)

In [ ]:

#from tensorflow.keras import layers

def create_classifiction_model(num_classes=196,IMG_SIZE=300):
  inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
  model = keras.applications.efficientnet.EfficientNetB3(include_top=False, input_tensor=inputs, weights="imagenet")

  # Freeze the pretrained weights
  model.trainable = False

  #for layer in model.layers[-20:]:
   # if not isinstance(layer, BatchNormalization):
    #  layer.trainable = True
  
  
  # Rebuild top
  x = GlobalAveragePooling2D(name="avg_pool")(model.output)
  x = BatchNormalization()(x)

  top_dropout_rate = 0.5
  x = Dropout(top_dropout_rate, name="top_dropout")(x)
  outputs = Dense(num_classes, activation="softmax", name="class_output")(x)

  return Model(inputs, outputs)

In [ ]:
def create_final_baseline_model(trainable=True):

    #inputToRegModel =Input((128,128,3))
    # Load weights from saved reg model
    regODModel=create_reg_od_model(False)
    regODModel.load_weights('/content/drive/MyDrive/Capstone/Pickled_Info/RegODModel_bestWieghts_128.h5')
    classModel=create_classifiction_model()
    classModel.load_weights('/content/drive/MyDrive/Capstone/Pickled_Info/classification_best_weights_300_sync.h5')


    #regODModel.compile(loss="mean_squared_error", optimizer="adam", metrics=[IoU]) # Regression loss is MSE
    return Model(inputs=[regODModel.input,classModel.input], outputs=[regODModel.output,classModel.output])
    #return regODModel



In [ ]:
# Define evaluation metric

def IOU(y_true, y_pred):
    intersections = 0
    unions = 0
    gt = y_true
    pred = y_pred
    # Compute interection of predicted (pred) and ground truth (gt) bounding boxes
    diff_width = np.minimum(gt[:,0] + gt[:,2], pred[:,0] + pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
    diff_height = np.minimum(gt[:,1] + gt[:,3], pred[:,1] + pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
    intersection = diff_width * diff_height

    # Compute union
    area_gt = gt[:,2] * gt[:,3]
    area_pred = pred[:,2] * pred[:,3]
    union = area_gt + area_pred - intersection
    # rk - again to repeat we know why he has minues , taken 2 times 

    # Compute intersection and union over multiple boxes
    for j, _ in enumerate(union):
      if union[j] > 0 and intersection[j] > 0 and union[j] >= intersection[j]:
        intersections += intersection[j]
        unions += union[j]

    # Compute IOU. Use epsilon to prevent division by zero
    iou = np.round(intersections / (unions + tensorflow.keras.backend.epsilon()), 4)# rounded to 4 
    # This must match the type used in py_func
    iou = iou.astype(np.float32)
    return iou
    # this is one number for the whole model

In [ ]:
def IoU(y_true, y_pred):
    iou = tensorflow.py_function(IOU, [y_true, y_pred], Tout=tensorflow.float32)
    return iou
# This is a wrapper funcction

In [ ]:
#Initialize the model and print summary

model = create_final_baseline_model(False) # Arg is False, if you want to freeze lower layers for fast training (but low accuracy)
model.summary() # Print summary

43950080/43941136 [==============================] - 1s 0us/step
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 300, 300, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 300, 300, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding

In [ ]:
# For classification we will have cateogirical crossentropy

# For the bouding boxes we will have mean squared error

losses = {

    "box_output":"mean_squared_error",

    "class_output": "categorical_crossentropy"

        }


# If you think that detection is harder then the classification then you can
# Try assinging it more weight
loss_weights = {

    "box_output": 1.0,

    "class_output": 1.0
                }


# Set the Metrics

# For the class labels we want to know the Accuracy
# And for the bounding boxes we need to know the Mean squared error

custommetrics = {

    "box_output": IoU,

    "class_output": 'accuracy'

        }


In [ ]:
#Compile the model

model.compile(loss=losses, optimizer='adam', metrics=custommetrics,loss_weights=loss_weights)
#model.compile(loss=losses, optimizer='SGD', metrics=custommetrics,loss_weights=loss_weights)

In [ ]:
#end

In [ ]:
############################################################# Load the class indices dictionary #####################################################################################

In [ ]:
import pickle

# open a file, where you stored the pickled data
file = open('/content/drive/MyDrive/Capstone/Pickled_Info/Classindices.pkl', 'rb')

# dump information to that file
classDict = pickle.load(file)

In [ ]:

# list out keys and values separately
key_list = list(classDict.keys())
val_list = list(classDict.values())
 
# print key with val 100
position = val_list.index(119)
print(key_list[position])

GMC Savana Van 2012


In [ ]:
# Take some sample image and predict

#region=model.predict(X_Test_MobileNet[2345].reshape(1,128,128,3)) # Predict the BBox

#image =X_Batch[10].reshape(1,300,300,3)
#image.shape
#region=model.predict({"input_1":image,"input_2":image}) # Predict the BBox

In [ ]:
# #########################################################Lets cacluate mAp on validation data set##########################################################

In [ ]:
# Have a look at validation data again 
valid_df.head()
valid_df.shape

,file_path,label,file_name,cor1,cor2,cor3,cor4,Image_Class
2733,/content/drive/MyDrive/Capstone/Car Images/Car...,Chrysler Sebring Convertible 2010,02306.jpg,29,95,557,375,77
4421,/content/drive/MyDrive/Capstone/Car Images/Car...,Daewoo Nubira Wagon 2002,07283.jpg,108,95,621,392,82
6320,/content/drive/MyDrive/Capstone/Car Images/Car...,Buick Verano Sedan 2012,02450.jpg,51,99,595,382,49
371,/content/drive/MyDrive/Capstone/Car Images/Car...,Volvo XC90 SUV 2007,02332.jpg,65,92,606,420,195
3937,/content/drive/MyDrive/Capstone/Car Images/Car...,Chrysler 300 SRT-8 2010,03784.jpg,84,150,1029,616,79


(1629, 8)

In [ ]:
images_width=128
images_height=128

In [ ]:
# Calculate mAp
import matplotlib.image as img
import cv2
from keras.applications.mobilenet import preprocess_input

#images_height # defined earlier
#images_width # defined earlier

groundTruth=[]
predictions=[]

# Loop over validation data
for i in np.arange(0,len(valid_df)-1,1):
  #Read each image  
  eachRecord=valid_df.iloc[i,:]
  im = img.imread(eachRecord['file_path'])
  imgLength=im.shape[0]
  imgWidth=im.shape[1]

  # Getting the ground truth
  x0=int(eachRecord['cor1'])
  y0=int(eachRecord['cor2'])
  x1=int(eachRecord['cor3'])
  y1=int(eachRecord['cor4'])
  imageClass=classDict.get(eachRecord['label'])

  # Resize Coordinates because each image will be 300*300
  res_x0=int(x0*images_width/imgWidth)
  res_y0=int(y0*images_height/imgLength)
  res_x1=int(x1*images_width/imgWidth)
  res_y1=int(y1*images_height/imgLength)
  #format required # [xmin, ymin, xmax, ymax, class_id, difficult, crowd]
  eachGroundTruth=(res_x0,res_y0,res_x1,res_y1,imageClass,0,0)

  #Resize image for regression n/w
  #im1=preprocess_input(im)
  resizedImage=cv2.resize(im, dsize=(images_height, images_width), interpolation=cv2.INTER_CUBIC)
  #Some These images had 4 channels , needed below line to convert to 3 
  convertedImage=cv2.cvtColor(resizedImage, cv2.COLOR_BGRA2BGR)
  im_1=convertedImage
  #Pre-process input for mobile net
  #im1=preprocess_input(im1)

  #Resize image for classification n/w
  #Resize image
  resizedImage=cv2.resize(im, dsize=(300, 300), interpolation=cv2.INTER_CUBIC)
  #Some These images had 4 channels , needed below line to convert to 3 
  convertedImage=cv2.cvtColor(resizedImage, cv2.COLOR_BGRA2BGR)
  im_2=convertedImage

  #Format needed by library is for gt is 
  ## [xmin, ymin, xmax, ymax, class_id, difficult, crowd]
  
  #Now for each prediction
  # For each image we will need to first resize
  
  im_1=preprocess_input(im_1)
  image1 =im_1.reshape(1,128,128,3)
  image2 =im_2.reshape(1,300,300,3)
  region=model.predict({"input_1":image1,"input_2":image2}) # Predict the BBox
  predEachBox=region[0][0]
  predEachClass=np.argmax(region[1][0])
  #print(predEachBox)
  #print(predEachClass)

  #[xmin, ymin, xmax, ymax, class_id, confidence]
  eachPrediction=(predEachBox[0],predEachBox[1],predEachBox[2],predEachBox[3],predEachClass,1)

  #Append resuts
  groundTruth.append(eachGroundTruth)
  predictions.append(eachPrediction)

In [ ]:
len(groundTruth)
print('\n')
len(predictions)

1628

1628

In [ ]:
######################################################### Calulate mAp ################################################################################

In [ ]:
! pip install mean_average_precision

In [ ]:
import numpy as np
from mean_average_precision import MetricBuilder

# [xmin, ymin, xmax, ymax, class_id, difficult, crowd]

gt =np.array(groundTruth)
preds=np.array(predictions)


# print list of available metrics
print(MetricBuilder.get_metrics_list())

# create metric_fn
metric_fn = MetricBuilder.build_evaluation_metric("map_2d", async_mode=True, num_classes=196)

# add some samples to evaluation
#for i in range(10):
metric_fn.add(preds, gt)

#metric_fn.add



# compute PASCAL VOC metric
#print(f"VOC PASCAL mAP: {metric_fn.value(iou_thresholds=0.5, recall_thresholds=np.arange(0., 1.1, 0.1))['mAP']}")

# compute PASCAL VOC metric at the all points
print(f"VOC PASCAL mAP in all points: {metric_fn.value(iou_thresholds=0.5)['mAP']}")

# compute metric COCO metric
#print(f"COCO mAP: {metric_fn.value(iou_thresholds=np.arange(0.5, 1.0, 0.05), recall_thresholds=np.arange(0., 1.01, 0.01), mpolicy='soft')['mAP']}")

['map_2d']
VOC PASCAL mAP in all points: 0.5921118855476379
